# setup

In [1]:
import torch
import os

In [2]:
if os.getcwd().split('/')[-1] == 'notebooks':
    os.chdir('..')

from data.load_data import LoadData
from models.baseline import ResnetBaseline

from models.moe import ResnetMoE
from utils import synthesis

# init

In [3]:
# model_label = 'gate'
model_label = 'moe'

In [4]:
if model_label == 'gate':
    from configs.gate import LoadDataConfig, Downstream_cnn_args

    loader_config = LoadDataConfig()
    resnet_config = Downstream_cnn_args()

    dataloader = LoadData(**loader_config.__dict__)
    reference = ResnetBaseline(**resnet_config.__dict__)

if model_label == 'moe':
    from configs.baseline import LoadDataConfig
    from configs.moe import MoE_cnn_args

    loader_config = LoadDataConfig()
    moe_config = MoE_cnn_args()

    dataloader = LoadData(**loader_config.__dict__)
    reference = ResnetMoE(**moe_config.__dict__)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# threshould

In [5]:
model = torch.load('output/conjugado_{}.pt'.format(model_label))
# assert model.state_dict().keys() == reference.state_dict().keys()
model = model.to(device)

In [6]:
val_dl = dataloader.get_val_dataloader()
best_f1s, best_thresholds = synthesis(model, val_dl, None, device)
best_f1s, best_thresholds

  0%|          | 0/272 [00:00<?, ?it/s]

6


100%|██████████| 272/272 [05:00<00:00,  1.11s/it]


6


([0.5619128949615713,
  0.8166281755196305,
  0.7946127946127947,
  0.6692975532754538,
  0.720658888126287,
  0.7494577006507592],
 [0.29, 0.37, 0.71, 0.34, 0.3, 0.32])

In [7]:
# import numpy as np
# from tqdm import tqdm
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# from utils import get_inputs

In [8]:
# num_classes = 6
# thresholds = np.arange(0, 1.01, 0.01)  # Array of thresholds from 0 to 1 with step 0.01
# predictions = {thresh: [[] for _ in range(num_classes)] for thresh in thresholds}
# true_labels_dict = [[] for _ in range(num_classes)]

In [9]:
# val_dl = dataloader.get_val_dataloader()

In [10]:
# model.eval()
# with torch.no_grad():
#     for val_batch in tqdm(val_dl):
#         raw, exam_id, label = val_batch
#         ecg = get_inputs(raw).to(device)
#         label = label.to(device).float()

#         logits = model(ecg)
#         probs = torch.sigmoid(logits)

#         for class_idx in range(num_classes):
#             for thresh in thresholds:
#                 predicted_binary = (probs[:, class_idx] >= thresh).float()
#                 predictions[thresh][class_idx].extend(
#                     predicted_binary.cpu().numpy()
#                 )
#             true_labels_dict[class_idx].extend(
#                 label[:, class_idx].cpu().numpy()
#             )

In [11]:
# best_thresholds = [0.5] * num_classes
# best_f1s = [0.0] * num_classes

# for class_idx in (range(num_classes)):
#     for thresh in thresholds:
#         f1 = f1_score(
#             true_labels_dict[class_idx],
#             predictions[thresh][class_idx],
#             zero_division=0,
#         )

#         if f1 > best_f1s[class_idx]:
#             best_f1s[class_idx] = f1
#             best_thresholds[class_idx] = thresh

In [12]:
# best_f1s, best_thresholds

# test

In [13]:
test_dl = dataloader.get_test_dataloader()
all_binary_results, all_true_labels, metrics_dict = synthesis(model, test_dl, best_thresholds, device)
metrics_dict

100%|██████████| 135/135 [03:35<00:00,  1.59s/it]

6


{'Accuracy': [0.9826927529520723,
  0.9877865246584857,
  0.9923593424403797,
  0.988075943505441,
  0.9882495948136143,
  0.9854132901134521],
 'Precision': [0.5245398773006135,
  0.7418300653594772,
  0.7703180212014135,
  0.5730994152046783,
  0.6895787139689579,
  0.6524822695035462],
 'Recall': [0.5428571428571428,
  0.8954635108481263,
  0.7649122807017544,
  0.765625,
  0.8315508021390374,
  0.8679245283018868],
 'F1 Score': [0.5335413416536662,
  0.8114387846291331,
  0.7676056338028169,
  0.6555183946488294,
  0.753939393939394,
  0.7449392712550608]}

In [14]:
# test_dl = dataloader.get_test_dataloader()

In [15]:
# all_binary_results = []
# all_true_labels = []

In [16]:
# model.eval()
# with torch.no_grad():
#     for test_batch in tqdm(test_dl):
#         raw, exam_id, label = test_batch
#         ecg = get_inputs(raw).to(device)
#         label = label.to(device).float()

#         logits = model(ecg)
#         probs = torch.sigmoid(logits)

#         binary_result = torch.zeros_like(probs)
#         for i in range(len(best_thresholds)):
#             binary_result[:, i] = (
#                 probs[:, i] >= best_thresholds[i]
#             ).float()

#         # Append binary results and true labels for this batch
#         all_binary_results.append(binary_result)
#         all_true_labels.append(label)
# all_binary_results = torch.cat(all_binary_results, dim=0)
# all_true_labels = torch.cat(all_true_labels, dim=0)

In [17]:
# accuracy_scores = []
# precision_scores = []
# recall_scores = []
# f1_scores = []

In [18]:
# for class_idx in range(num_classes):
#     class_binary_results = all_binary_results[:, class_idx].cpu().numpy()
#     class_true_labels = all_true_labels[:, class_idx].cpu().numpy()

#     accuracy = accuracy_score(class_true_labels, class_binary_results)
#     precision = precision_score(
#         class_true_labels, class_binary_results, zero_division=0
#     )
#     recall = recall_score(
#         class_true_labels, class_binary_results, zero_division=0
#     )
#     f1 = f1_score(class_true_labels, class_binary_results, zero_division=0)

#     accuracy_scores.append(accuracy)
#     precision_scores.append(precision)
#     recall_scores.append(recall)
#     f1_scores.append(f1)

# metrics_dict = {
#     "Class": dataloader.output_col,
#     "Accuracy": accuracy_scores,
#     "Precision": precision_scores,
#     "Recall": recall_scores,
#     "F1 Score": f1_scores,
# }

In [19]:
# metrics_dict